This notebook is used just in order to play around in Python and try different things.

In [2]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
# convert an address into latitude and longitude values
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
!pip install matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
!pip install folium
import folium

# library to handle JSON files
import json

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

import xml.etree.ElementTree as et 

print("Done!")

Done!


The geograpical coordinate of Stockholm are 59.3251172, 18.0710935.


Let's scrape a table from Wikipedia that contains information about the metro stations:

In [30]:
source=requests.get('https://sv.wikipedia.org/w/index.php?title=Lista_%C3%B6ver_tunnelbanestationer_i_Stockholm&oldid=46033661').text
soup = BeautifulSoup(source,'lxml')
table=soup.find('table',{'class':'sortable wikitable'})
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

df = pd.DataFrame(res, columns=["Station", "Line", "Year", "Distance", "Municipality", "Type", "Longitude", "Latitude"])

In [31]:
df.head(110)

,Station,Line,Year,Distance,Municipality,Type,Longitude,Latitude
0,Slussen,Grön,1950,"1,5",Stockholms kommun,Betongstation,18.072327,59.319493
1,Medborgarplatsen,Grön,1950,"2,1",Stockholms kommun,Betongstation,18.073550,59.314342
2,Skanstull,Grön,1950,"2,7",Stockholms kommun,Betongstation,18.076229,59.307852
3,Gullmarsplan,Grön,1950,"3,8",Stockholms kommun,"Ytstation, delvis överbyggd av betong",18.080768,59.299114
4,Skärmarbrink,Grön,1950,"4,5",Stockholms kommun,Ytstation,18.090440,59.295366
5,Blåsut,Grön,1950,"5,1",Stockholms kommun,Ytstation,18.091061,59.290242
6,Sandsborg,Grön,1950,"5,7",Stockholms kommun,Ytstation,18.092382,59.284785
7,Skogskyrkogården,Grön,1950,"6,5",Stockholms kommun,Ytstation,18.095501,59.279194
8,Tallkrogen,Grön,1950,"7,5",Stockholms kommun,Ytstation,18.085326,59.271140
9,Gubbängen,Grön,1950,"8,2",Stockholms kommun,Ytstation,18.082036,59.262879


Let's create a new dataframe where:
* Only opened och functioning stations are left (i.e. I remove Kymlinge that was never opened)
* All columns that are not going to be used are skipped (i.e. I keep only the station name, longitude and latitude)
* Dublicates are removed (in the original table there are some stations that are represented on both lines but since the fact that those stations have the same name and coordinates and lines are not of interest for this project, I'll keep only one copy of those)
* Information about previous names is removed since it's not relevant for this assigment

In [32]:
df1=df[df['Year'].astype(str).str.isdigit()]
df1
df2=df1[['Station','Longitude','Latitude']]
df2
df2.describe()
df3=df2.drop_duplicates(["Station"]).reset_index(drop=True)
df3.describe()
df4=df3
df4['Station'] = df3['Station'].str.replace(r" \(.*\)","")
df4

,Station,Longitude,Latitude
0,Slussen,18.072327,59.319493
1,Medborgarplatsen,18.073550,59.314342
2,Skanstull,18.076229,59.307852
3,Gullmarsplan,18.080768,59.299114
4,Skärmarbrink,18.090440,59.295366
5,Blåsut,18.091061,59.290242
6,Sandsborg,18.092382,59.284785
7,Skogskyrkogården,18.095501,59.279194
8,Tallkrogen,18.085326,59.271140
9,Gubbängen,18.082036,59.262879


Now let's put the stations on the map of Stockholm.

In [33]:
address = 'Stockholm, Sweden'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Stockholm are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Stockholm are 59.3251172, 18.0710935.


In [35]:
# create map of Stockholm
map_sthlm = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df4['Latitude'], df4['Longitude'], df4['Station']):
#    label = '{}'.format(station)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sthlm)  
    
map_sthlm

In [52]:
CLIENT_ID = 'RWLY0RHFLLB2X5L5SF2FV243QOFFSINLN0HZUOT1BFEBCK5G' # your Foursquare ID
CLIENT_SECRET = 'SBFSJDUVKAH2EJRJFRDS1ZYIQYLTKOTXAYREQN5AENT4L5PA' # your Foursquare Secret
VERSION = '20190703' # Foursquare API version

Let's run a short analysis of the appropriate distance between the stations. Or just describe why I pick 500 m.

In [71]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Metro station', 
                  'Metro station Latitude', 
                  'Metro station Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [74]:
sthlm_venues = getNearbyVenues(names=df4['Station'],
                                   latitudes=df4['Latitude'],
                                   longitudes=df4['Longitude']
                                  )

Slussen
Medborgarplatsen
Skanstull
Gullmarsplan
Skärmarbrink
Blåsut
Sandsborg
Skogskyrkogården
Tallkrogen
Gubbängen
Hökarängen
Globen
Enskede gård
Sockenplan
Svedmyra
Stureby
Hötorget
Rådmansgatan
Odenplan
Sankt Eriksplan
Fridhemsplan
Thorildsplan
Kristineberg
Alvik
Stora mossen
Abrahamsberg
Brommaplan
Åkeshov
Ängbyplan
Islandstorget
Blackeberg
Råcksta
Vällingby
Bandhagen
Högdalen
Johannelund
Hässelby gård
T-Centralen
Gamla stan
Hammarbyhöjden
Björkhagen
Kärrtorp
Bagarmossen
Farsta
Hässelby strand
Rågsved
Hagsätra
Mariatorget
Zinkensdamm
Hornstull
Liljeholmen
Midsommarkransen
Telefonplan
Hägerstensåsen
Västertorp
Fruängen
Aspudden
Örnsberg
Axelsberg
Mälarhöjden
Bredäng
Sätra
Östermalmstorg
Karlaplan
Gärdet
Ropsten
Skärholmen
Vårberg
Farsta strand
Vårby gård
Masmo
Fittja
Stadion
Tekniska högskolan
Universitetet
Alby
Hallunda
Norsborg
Rådhuset
Stadshagen
Västra skogen
Solna centrum
Näckrosen
Hallonbergen
Rinkeby
Tensta
Hjulsta
Kista
Husby
Akalla
Kungsträdgården
Bergshamra
Danderyds sjukh

In [148]:
sthlm_venues.shape
# with limit = 100 I get (2392,7) here

(2392, 7)

In [149]:
#sthlm_venues.head(20)

In [150]:
sthlm_venues.groupby('Metro station').count()

,Metro station Latitude,Metro station Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Metro station,,,,,,
Abrahamsberg,4,4,4,4,4,4
Akalla,14,14,14,14,14,14
Alby,6,6,6,6,6,6
Alvik,19,19,19,19,19,19
Aspudden,14,14,14,14,14,14
Axelsberg,4,4,4,4,4,4
Bagarmossen,6,6,6,6,6,6
Bandhagen,11,11,11,11,11,11
Bergshamra,11,11,11,11,11,11


In [151]:
print('There are {} uniques categories.'.format(len(sthlm_venues['Venue Category'].unique())))

There are 257 uniques categories.


In [168]:
# one hot encoding
sthlm_onehot = pd.get_dummies(sthlm_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
sthlm_onehot['Metro station'] = sthlm_venues['Metro station'] 

# move neighbourhood column to the first column
fixed_columns = [sthlm_onehot.columns[-1]] + list(sthlm_onehot.columns[:-1])
sthlm_onehot = sthlm_onehot[fixed_columns]

sthlm_onehot.head()

,Metro station,Accessories Store,Advertising Agency,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Camera Store,Candy Store,Caribbean Restaurant,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Community Center,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Currency Exchange,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Lebanese Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nature Preserve,Neighborhood,Newsstand,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Gym,Outdoor Sculpture,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Soccer Field,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Slussen,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [169]:
sthlm_onehot.shape

(2392, 258)

In [170]:
sthlm_grouped = sthlm_onehot.groupby('Metro station').mean().reset_index()
sthlm_grouped.head()

,Metro station,Accessories Store,Advertising Agency,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Camera Store,Candy Store,Caribbean Restaurant,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comic Shop,Community Center,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Currency Exchange,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dive Bar,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Lebanese Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,Nature Preserve,Neighborhood,Newsstand,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Gym,Outdoor Sculpture,Outlet Store,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Soccer Field,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Abrahamsberg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [171]:
sthlm_grouped.shape

(100, 258)

In [172]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

consider removing metro stations from the top 10 categories

In [189]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Metro station']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
metro_stations_venues_sorted = pd.DataFrame(columns=columns)
metro_stations_venues_sorted['Metro station'] = sthlm_grouped['Metro station']

for ind in np.arange(sthlm_grouped.shape[0]):
    metro_stations_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sthlm_grouped.iloc[ind, :], num_top_venues)

metro_stations_venues_sorted.head()

,Metro station,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abrahamsberg,Pizza Place,Gym / Fitness Center,Steakhouse,Video Store,Yoga Studio,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
1,Akalla,Farm,Asian Restaurant,Supermarket,Grocery Store,Candy Store,Metro Station,Plaza,Middle Eastern Restaurant,Convenience Store,Pizza Place
2,Alby,Metro Station,Football Stadium,Pizza Place,Convenience Store,Bus Stop,Shopping Mall,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
3,Alvik,Hotel,Pizza Place,Grocery Store,Burger Joint,Modern European Restaurant,Metro Station,Furniture / Home Store,Scandinavian Restaurant,Bistro,Martial Arts Dojo
4,Aspudden,Thai Restaurant,Grocery Store,Convenience Store,Golf Course,Park,Pizza Place,Café,Bookstore,Sushi Restaurant,Bakery


In [190]:
# Here I should go back to the previos course and check how to find best k (if it's the same k)
# set number of clusters
kclusters = 5

sthlm_grouped_clustering = sthlm_grouped.drop('Metro station', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sthlm_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([0, 2, 1, 2, 2, 0, 0, 1, 0, 0, 2, 2, 0, 2, 1, 0, 1, 2, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 1, 1, 1, 0, 4, 2, 2, 0, 2, 2, 1, 2, 2, 2, 0, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 0, 1, 2, 2, 1, 2, 2, 0, 2, 2, 1, 0,
       2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 0, 2, 0, 2, 2, 1,
       2, 2, 2, 0, 2, 1, 0, 2, 2, 2, 0, 2])

In [191]:
#metro_stations_venues_sorted

In [192]:
#sthlm_merged

In [193]:
# add clustering labels
metro_stations_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sthlm_merged = df4

# merge toronto_grouped with toronto_data to add latitude/longitude for each metro station
sthlm_merged = sthlm_merged.join(metro_stations_venues_sorted.set_index('Metro station'), on='Station')

sthlm_merged.head()

,Station,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Slussen,18.072327,59.319493,2,Scandinavian Restaurant,Café,Pub,Bar,Bakery,Boat or Ferry,Cocktail Bar,Arts & Crafts Store,Record Shop,Furniture / Home Store
1,Medborgarplatsen,18.073550,59.314342,2,Bar,Clothing Store,Café,Record Shop,Scandinavian Restaurant,Bookstore,Gym / Fitness Center,Furniture / Home Store,Coffee Shop,Indian Restaurant
2,Skanstull,18.076229,59.307852,2,Café,Thai Restaurant,Nightclub,Beer Bar,Gym,Sushi Restaurant,Dumpling Restaurant,Pizza Place,Seafood Restaurant,Playground
3,Gullmarsplan,18.080768,59.299114,2,Coffee Shop,Convenience Store,Eastern European Restaurant,Basketball Court,Bar,Flower Shop,Greek Restaurant,Grocery Store,BBQ Joint,Fast Food Restaurant
4,Skärmarbrink,18.090440,59.295366,2,Hotel,Concert Hall,Hockey Arena,Eastern European Restaurant,Thai Restaurant,Yoga Studio,Flea Market,Farm,Farmers Market,Fast Food Restaurant


In [194]:
sthlm_merged.groupby('Cluster Labels').count()

,Station,Longitude,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,
0,19,19,19,19,19,19,19,19,19,19,19,19,19
1,19,19,19,19,19,19,19,19,19,19,19,19,19
2,60,60,60,60,60,60,60,60,60,60,60,60,60
3,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1


In [195]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sthlm_merged['Latitude'], sthlm_merged['Longitude'], sthlm_merged['Station'], sthlm_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [163]:
sthlm_merged.loc[sthlm_merged['Cluster Labels'] == 0, sthlm_merged.columns[[0] + list(range(3, sthlm_merged.shape[1]))]]

,Station,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Slussen,0,Scandinavian Restaurant,Café,Pub,Bar,Bakery,Boat or Ferry,Cocktail Bar,Arts & Crafts Store,Record Shop,Furniture / Home Store
1,Medborgarplatsen,0,Bar,Clothing Store,Café,Record Shop,Scandinavian Restaurant,Bookstore,Gym / Fitness Center,Furniture / Home Store,Coffee Shop,Indian Restaurant
2,Skanstull,0,Café,Thai Restaurant,Nightclub,Beer Bar,Gym,Sushi Restaurant,Dumpling Restaurant,Pizza Place,Seafood Restaurant,Playground
3,Gullmarsplan,0,Coffee Shop,Convenience Store,Eastern European Restaurant,Basketball Court,Bar,Flower Shop,Greek Restaurant,Grocery Store,BBQ Joint,Fast Food Restaurant
4,Skärmarbrink,0,Hotel,Concert Hall,Hockey Arena,Eastern European Restaurant,Thai Restaurant,Yoga Studio,Flea Market,Farm,Farmers Market,Fast Food Restaurant
5,Blåsut,0,Pizza Place,Thai Restaurant,Gym / Fitness Center,Bus Stop,Stadium,Gay Bar,Scandinavian Restaurant,Chinese Restaurant,Nightclub,Bar
7,Skogskyrkogården,0,Café,Paper / Office Supplies Store,Cemetery,Thai Restaurant,Arts & Crafts Store,Bakery,Food & Drink Shop,Food Court,Food Truck,Football Stadium
10,Hökarängen,0,Grocery Store,Convenience Store,Plaza,Bakery,Shopping Plaza,Café,Coffee Shop,Pizza Place,Scandinavian Restaurant,Flower Shop
11,Globen,0,Nightclub,Concert Hall,Gym / Fitness Center,Hotel,Scenic Lookout,General Entertainment,Liquor Store,Supermarket,Music Venue,Chinese Restaurant
12,Enskede gård,0,Stables,Metro Station,Bakery,Farm,French Restaurant,Fraternity House,Football Stadium,Food Truck,Food Court,Food & Drink Shop


In [164]:
sthlm_merged.loc[sthlm_merged['Cluster Labels'] == 1, sthlm_merged.columns[[0] + list(range(3, sthlm_merged.shape[1]))]]

,Station,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Hässelby strand,1,Thai Restaurant,Grocery Store,Plaza,Metro Station,Bus Stop,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
45,Rågsved,1,Bus Station,Metro Station,Grocery Store,Diner,Yoga Studio,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
46,Hagsätra,1,Shopping Mall,Grocery Store,Gym / Fitness Center,Food & Drink Shop,Food,Metro Station,Yoga Studio,Farm,Farmers Market,Fast Food Restaurant
60,Bredäng,1,Grocery Store,Pizza Place,Bus Stop,Gym,Gym / Fitness Center,Metro Station,Convenience Store,Food & Drink Shop,Food,Flower Shop
67,Vårberg,1,Grocery Store,Metro Station,Flea Market,Shopping Mall,Event Space,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Food
71,Fittja,1,Shopping Mall,Bus Station,Supermarket,Metro Station,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Yoga Studio
75,Alby,1,Metro Station,Football Stadium,Pizza Place,Convenience Store,Bus Stop,Shopping Mall,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
83,Hallonbergen,1,Grocery Store,Lake,Gym / Fitness Center,Bus Stop,Café,Shopping Mall,Metro Station,Tennis Court,Thai Restaurant,Gourmet Shop
84,Rinkeby,1,Metro Station,Grocery Store,Convenience Store,Bus Stop,Shopping Mall,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
85,Tensta,1,Indian Restaurant,Art Gallery,Bus Stop,Metro Station,Supermarket,Shopping Mall,Grocery Store,Department Store,Design Studio,Football Stadium


In [165]:
sthlm_merged.loc[sthlm_merged['Cluster Labels'] == 2, sthlm_merged.columns[[0] + list(range(3, sthlm_merged.shape[1]))]]

,Station,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Sandsborg,2,Scandinavian Restaurant,Gym / Fitness Center,Pizza Place,Convenience Store,Playground,Bus Stop,Grocery Store,Metro Station,Yoga Studio,Falafel Restaurant
8,Tallkrogen,2,Indian Restaurant,Metro Station,Gym / Fitness Center,Grocery Store,Pizza Place,Arcade,Farm,Fried Chicken Joint,French Restaurant,Fraternity House
25,Abrahamsberg,2,Pizza Place,Gym / Fitness Center,Steakhouse,Video Store,Yoga Studio,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
29,Islandstorget,2,Chinese Restaurant,Bus Stop,Bakery,Metro Station,Soccer Field,Pizza Place,Food Court,Food Truck,Football Stadium,Food & Drink Shop
31,Råcksta,2,Metro Station,Soccer Field,Pizza Place,Furniture / Home Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Yoga Studio
39,Hammarbyhöjden,2,Thai Restaurant,Pizza Place,Soccer Field,Bakery,Flea Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
40,Björkhagen,2,Sushi Restaurant,Metro Station,Grocery Store,Gym / Fitness Center,Nature Preserve,Bakery,Pizza Place,Food,Food & Drink Shop,Flower Shop
57,Örnsberg,2,Pizza Place,Bus Stop,Bakery,Grocery Store,Golf Course,Park,Café,Thai Restaurant,Chinese Restaurant,Metro Station
58,Axelsberg,2,Pizza Place,Neighborhood,Metro Station,Grocery Store,Czech Restaurant,Farm,Football Stadium,Food Truck,Food Court,Food & Drink Shop
61,Sätra,2,Pizza Place,Middle Eastern Restaurant,Metro Station,Grocery Store,Food Truck,Football Stadium,Food Court,Fraternity House,Falafel Restaurant,Food


In [166]:
sthlm_merged.loc[sthlm_merged['Cluster Labels'] == 3, sthlm_merged.columns[[0] + list(range(3, sthlm_merged.shape[1]))]]

,Station,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Gubbängen,3,Indian Restaurant,Music Venue,Scandinavian Restaurant,Currency Exchange,Falafel Restaurant,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Food


In [167]:
sthlm_merged.loc[sthlm_merged['Cluster Labels'] == 4, sthlm_merged.columns[[0] + list(range(3, sthlm_merged.shape[1]))]]

,Station,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


Now let's add word "T-bana" to make our search in the next step more specific and therefore avoid a lot of cleaning afterwards:

In [176]:
#df4=df3
#df4['Station']=df4['Station'] + ' T-bana'
#df4

In [177]:
#df7=df4.iloc[90:120, :]
#df7

Now let's send an API request to get the coordinates for every metro station:

In [178]:
#url='https://api.resrobot.se/v2/location.name?key=57784d15-1b13-49bd-9fea-57c5d71af270&input=Gamla%20stan%20T-bana&format=json'
#response=requests.get(url).json()
#response

Let's keep all of the columns to begin with in order to use this information to make some checks that the data quality is OK:

In [179]:
#df5=response['StopLocation']
#df6 = json_normalize(df5)
#df6.head()

Let's create a list of metro stations that will be used for API call:

In [180]:
#stations=list(df7["Station"])
#stations

In [181]:
#response['StopLocation'][0]['id']

In [172]:
#st_list = []

# For each metro station, we access its information through its API
#for Station in stations:
 #   JSONContent = requests.get("https://api.resrobot.se/v2/location.name?key=57784d15-1b13-49bd-9fea-57c5d71af270&format=json&input=" + Station).json()
  #  st_list.append([JSONContent['StopLocation'][0]['id'], JSONContent['StopLocation'][0]['extId'], JSONContent['StopLocation'][0]['name'],
   #                      JSONContent['StopLocation'][0]['lon'], JSONContent['StopLocation'][0]['lat'], JSONContent['StopLocation'][0]['weight'], JSONContent['StopLocation'][0]['products']])
                         
#dataset = pd.DataFrame(st_list)
#dataset

In [173]:
#dataset

In [1]:
# ruby grab_venues.rb will print a csv of all the foursquare venue types
# flattens the nested hash of venues

url = "https://api.foursquare.com/v2/venues/categories?oauth_token=your_token&v=20120828"

require 'json'

# hashes are nested, containing arrays of hashes of category information
def print_hash(hash)
  # We've found a category
  if hash['id'] and hash['name']
    puts "#{hash['id']},#{hash['name']}"
  end
  # We've found an array of hashes of categories (that might contain more arrays!)
  if hash["categories"]
    hash["categories"].each do |h|
      print_hash(h)
    end
  end
end

ret = `curl #{url}`
json_response = JSON.parse(ret)

puts json_response["response"]["categories"].first

json_response["response"]["categories"].each do |c|
  print_hash(c)
end

SyntaxError: invalid syntax (<ipython-input-1-e8a07b116e6f>, line 6)

In [175]:
#JSONContent

In [ ]:
#url='https://api.resrobot.se/v2/location.name?key=57784d15-1b13-49bd-9fea-57c5d71af270&input=Gamla%20stan%20T-bana'
#response=requests.get(url).
#print(response)

In [174]:
#df4['Station']

In [12]:
#map_sthlm = folium.Map(location=[latitude, longitude], zoom_start=10)
#map_sthlm

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, postcode, borough, neighbourhood in zip(df3['Latitude'], df3['Longitude'], df3['Postcode'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}, {}'.format(postcode, borough, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Explore boroughs that contain the word "Toronto"

Before I slice the data, I would like to know how many and which boroughs there are in the dataset and how many different postcodes there are in each of them:

In [7]:
df3.groupby('Borough')['Postcode'].nunique()

Borough
Central Toronto      9
Downtown Toronto    18
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Queen's Park         1
Scarborough         17
West Toronto         6
York                 5
Name: Postcode, dtype: int64

 Now I create a new dataframe (b_toronto) with only boroughs that contain the word Toronto:

In [8]:
b_toronto= df3 [df3['Borough'].str.contains("Toronto") == True].reset_index(drop=True)
b_toronto.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


I expect the dataset to have 9+18+5+6=38 rows and 5 columns:

In [9]:
b_toronto.shape

(38, 5)

Now I create a new map in order to visualize the new dataframe:

In [10]:
# create map of boroughs that include the word Toronto using latitude and longitude values
map_b_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(b_toronto['Latitude'], b_toronto['Longitude'], b_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_b_toronto)  
    
map_b_toronto

To define Foursquare credentials and version:

To create a function that gets venues for all the boroughs in b_toronto: 

To run the function above on each neighbourhood in the b_toronto dataset:

In [13]:
b_toronto_venues = getNearbyVenues(names=b_toronto['Neighbourhood'],
                                   latitudes=b_toronto['Latitude'],
                                   longitudes=b_toronto['Longitude']
                                  )


The Beaches


NameError: name 'LIMIT' is not defined

To check the size and first 20 rows of the resulting dataframe:

In [ ]:
b_toronto_venues.shape

In [ ]:
b_toronto_venues.head(20)

To check how many venues were returned for each neighbourhood:

In [ ]:
b_toronto_venues.groupby('Neighbourhood').count()

To find out how many unique categories can be curated from all the returned venues:

In [ ]:
print('There are {} uniques categories.'.format(len(b_toronto_venues['Venue Category'].unique())))

To analyze each neighbourhood:

In [ ]:
# one hot encoding
b_toronto_onehot = pd.get_dummies(b_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
b_toronto_onehot['Neighbourhood'] = b_toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [b_toronto_onehot.columns[-1]] + list(b_toronto_onehot.columns[:-1])
b_toronto_onehot = b_toronto_onehot[fixed_columns]

b_toronto_onehot.head()

To check the dataframe's size:

In [ ]:
b_toronto_onehot.shape

To group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category:

In [ ]:
b_toronto_grouped = b_toronto_onehot.groupby('Neighbourhood').mean().reset_index()
b_toronto_grouped.head()

The new size:

In [ ]:
b_toronto_grouped.shape

In [ ]:
b_toronto_grouped.head()

To return top 8 venues (just to try something else than 10) for each group of neighbourhoods:

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 8

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = b_toronto_grouped['Neighbourhood']

for ind in np.arange(b_toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(b_toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head(40)

### Cluster Neighbourhoods
I've tried to use different number of clusters and decided to keep the version with 5 clusters here:

In [ ]:
# set number of clusters
kclusters = 5

b_toronto_grouped_clustering = b_toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(b_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

To create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood:

In [ ]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

b_toronto_merged = b_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
b_toronto_merged = b_toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

b_toronto_merged.head()

To visualize the result:

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(b_toronto_merged['Latitude'], b_toronto_merged['Longitude'], b_toronto_merged['Neighbourhood'], b_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Next I'll examine each cluster and try to make some conclusions at the end of this notebook.

In [ ]:
b_toronto_merged.loc[b_toronto_merged['Cluster Labels'] == 0, b_toronto_merged.columns[[1] + list(range(5, b_toronto_merged.shape[1]))]]

In [ ]:
b_toronto_merged.loc[b_toronto_merged['Cluster Labels'] == 1, b_toronto_merged.columns[[1] + list(range(5, b_toronto_merged.shape[1]))]]

In [ ]:
b_toronto_merged.loc[b_toronto_merged['Cluster Labels'] == 2, b_toronto_merged.columns[[1] + list(range(5, b_toronto_merged.shape[1]))]]

In [ ]:
b_toronto_merged.loc[b_toronto_merged['Cluster Labels'] == 3, b_toronto_merged.columns[[1] + list(range(5, b_toronto_merged.shape[1]))]]

In [ ]:
b_toronto_merged.loc[b_toronto_merged['Cluster Labels'] == 4, b_toronto_merged.columns[[1] + list(range(5, b_toronto_merged.shape[1]))]]

### Some conclusions

The purpose of the exercise was to explore Toronto and some of its neighbourhoods, to try out clustering and to visualize the results.

The clustering itself would need to be improved in order to be able to describe the clusters in a meaningful way since 4 out of 5 clusters contain just one or two neighbourhoods while one of the clusters contains almost all the neighbourhoods. Since this kind of improvements are outside the scope of the task, I'll stop here.

Thank you for reviewing this notebook!